# PCA on Covariates Simulations Notebook

In [5]:
# Import objects from the setup file
import os
import sys
sys.path.append(os.path.expanduser('~/repo/ECMA-31330-Project/Source'))
from ME_Setup import *

# Packages
import numpy as np
import pandas as pd
import seaborn as sns
from pca import pca
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FactorAnalysis
import math
from tqdm import tqdm
import regex as re

In [6]:
# Supressing Output
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [7]:
# Data frame to store output
output = pd.DataFrame()

# 3,000 observations
N = 3000

# Loop over combinations of: betas, covariances between variables, p numbers of parameters
for beta1 in [0.1,1,10]:
    for beta2 in [0.1,1,10]:
        for covariance in [-0.9,-0.5,0,0.5,0.9]:
            for p in [5,20,50]:
                # Select only the scenarios we actually want to run by default - betas of 1, covariances of 0.5, p of 5... require three of these conditions to be satisfied to run.
                counter = 0
                if beta1 == 1:
                    counter += 1
                if beta2 == 1:
                    counter += 1
                if covariance == 0.5:
                    counter += 1
                if p == 5:
                    counter+=1
                if counter >= 3:
                    # Run with and without transformations
                    for exp_of_var in ['yes','no']:
                        # 1000 simulations
                        for k in tqdm(range(1000)):
                            # Initialize Lists to store coef values for all five methods and the true coef
                            pca_coef = []
                            mismeasured_coef = []
                            mismeasured_allvar_coef = []
                            mismeasured_avg_coef = []
                            iv_coef = []
                            true_val_coef =[]

                            # Create variables
                            vars_mean = [0,0,0]
                            vars_cov = np.array([[1,covariance,0],
                                                 [covariance,1,0],
                                                 [0,0,1]])
                            # Producing 3 variables: x for the variable of interest, the true Z covariate, the random error
                            vars_ = pd.DataFrame(np.random.multivariate_normal(vars_mean, vars_cov, N), columns = ['x','true_z','u'])
                            vars_['y'] = beta1 * vars_['x'] + beta2 * vars_['true_z'] + vars_['u']

                            # Create measurement errors for each of the p measurements of the covariates- mean zero and variance one
                            errors_mean = np.zeros(p)
                            errors_cov = np.zeros((p,p))
                            for i in range(p):
                                for j in range(p):
                                    if i == j:
                                        errors_cov[i,j] = 1

                            errors = np.random.multivariate_normal(errors_mean, errors_cov, N)
                            # Column labels for Z variables (covariates variables mismeasured)
                            z_vars = []
                            for i in range(p):
                                z_vars.append('z'+str(i+1))
                            # Add errors to the true_z to get mismeasured values
                            mismeasured_z = pd.DataFrame(errors, columns = z_vars)
                            for i in mismeasured_z.columns:
                                mismeasured_z[i] = mismeasured_z[i] + vars_['true_z']

                            # Take e to the power of the values for half of the measurements if log_of_var is true
                            if exp_of_var == 'yes':
                                mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):] =np.exp(mismeasured_z.iloc[:,int(len(mismeasured_z.columns)/2):])
                                
                            # Do feature scaling (normalize to mean 0 and variance 1) for the mismeasured z
                            # Note that x and y are already normalized by construction
                            scaled_mismeasured_z = mismeasured_z.copy()
                            for i in mismeasured_z.columns:
                                scaled_mismeasured_z[i] = (mismeasured_z[i] - mismeasured_z[i].mean()) / mismeasured_z[i].std()

                            # Suppress output
                            with suppress_stdout():
                                # Use PCA on the mismeasured values
                                pca_model = pca()
                                pca_results = pca_model.fit_transform(scaled_mismeasured_z)
                                pca_z = pca_results['PC']['PC1']

                            # NOTE: in non-pca cases, no need to rescale or normalize since mismeasured variables and x and y have mean 0 and sd 1

                            # Average mismeasured variables:
                            vars_['avg_mismeasured_z'] = mismeasured_z[z_vars].mean(axis=1)

                            # Add relevant variables to vars_ dataframe
                            vars_[mismeasured_z.columns] = mismeasured_z
                            vars_['pca_z'] = pca_z

                            # Single mismeasured covariate results
                            model_mismeasured = sm.OLS(vars_['y'],vars_[['x','z1']])
                            results_mismeasured = model_mismeasured.fit()
                            mismeasured_coef.append(results_mismeasured.params[0])

                            # All Variables Mismeasured Results
                            # Create full list of items to include in regression
                            tot_vars = ['x']
                            tot_vars.extend(z_vars)
                            model_mismeasured_allvar = sm.OLS(vars_['y'],vars_[tot_vars])
                            results_mismeasured_allvar = model_mismeasured_allvar.fit()
                            mismeasured_allvar_coef.append(results_mismeasured_allvar.params[0])

                            # Average Mismeasured Variables Results
                            model_mismeasured_avg = sm.OLS(vars_['y'],vars_[['x','avg_mismeasured_z']])
                            results_mismeasured_avg = model_mismeasured_avg.fit()
                            mismeasured_avg_coef.append(results_mismeasured_avg.params[0])

                            # PCA Results
                            model_pca = sm.OLS(vars_['y'],vars_[['x','pca_z']])
                            results_pca = model_pca.fit()
                            pca_coef.append(results_pca.params[0])

                            # Instrumental Variables Results
                            # Instrument z1 on the other items in the mismeasured df
                            z_string = mismeasured_z.columns[1]
                            for z in mismeasured_z.columns[2:]:
                                z_string = z_string + ' + ' + z
                            # Create the predicted value of z1
                            vars_['pred_z1'] = smf.ols("z1 ~ " + z_string, data = mismeasured_z).fit().predict()
                            iv_results = smf.ols("y ~ x + pred_z1 -1", data = vars_).fit()
                            iv_coef.append(iv_results.params[0])

                            # True Results
                            model_true = sm.OLS(vars_['y'],vars_[['x','true_z']])
                            results_true = model_true.fit()
                            true_val_coef.append(results_true.params[0])

                            # Output Findings
                            new_output = pd.DataFrame()
                            new_output['mismeasured_coef'] = mismeasured_coef
                            new_output['mismeasured_allvar_coef'] = mismeasured_allvar_coef
                            new_output['mismeasured_avg_coef'] = mismeasured_avg_coef
                            new_output['pca_coef'] = pca_coef
                            new_output['true_val_coef'] = true_val_coef
                            new_output['iv_coef'] = iv_coef
                            new_output['covariance'] = vars_cov[0][1]
                            new_output['beta1'] = beta1
                            new_output['beta2'] = beta2
                            new_output['p'] = p
                            new_output['exp_of_var'] = exp_of_var
                            output = output.append(new_output)

output

100%|██████████| 1000/1000 [12:03<00:00,  1.38it/s]


,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.394401,0.298674,0.476207,0.320473,0.122544,0.349810,0.5,0.1,1.0,5,yes
0,0.382212,0.270921,0.465710,0.301380,0.101579,0.332573,0.5,0.1,1.0,5,yes
0,0.343535,0.245985,0.441854,0.288636,0.067655,0.307192,0.5,0.1,1.0,5,yes
0,0.356660,0.254967,0.418646,0.275167,0.085243,0.306944,0.5,0.1,1.0,5,yes
0,0.375053,0.288823,0.471625,0.312702,0.101265,0.351331,0.5,0.1,1.0,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,10.325513,10.150916,10.150416,10.150195,10.045949,10.173591,0.5,10.0,1.0,5,no
0,10.318083,10.137120,10.138374,10.138453,10.030360,10.161507,0.5,10.0,1.0,5,no
0,10.310895,10.136993,10.136301,10.136207,10.025046,10.156673,0.5,10.0,1.0,5,no
0,10.301221,10.114495,10.115675,10.115735,10.007082,10.127878,0.5,10.0,1.0,5,no


In [8]:
output

,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,covariance,beta1,beta2,p,exp_of_var
0,0.394401,0.298674,0.476207,0.320473,0.122544,0.349810,0.5,0.1,1.0,5,yes
0,0.382212,0.270921,0.465710,0.301380,0.101579,0.332573,0.5,0.1,1.0,5,yes
0,0.343535,0.245985,0.441854,0.288636,0.067655,0.307192,0.5,0.1,1.0,5,yes
0,0.356660,0.254967,0.418646,0.275167,0.085243,0.306944,0.5,0.1,1.0,5,yes
0,0.375053,0.288823,0.471625,0.312702,0.101265,0.351331,0.5,0.1,1.0,5,yes
...,...,...,...,...,...,...,...,...,...,...,...
0,10.325513,10.150916,10.150416,10.150195,10.045949,10.173591,0.5,10.0,1.0,5,no
0,10.318083,10.137120,10.138374,10.138453,10.030360,10.161507,0.5,10.0,1.0,5,no
0,10.310895,10.136993,10.136301,10.136207,10.025046,10.156673,0.5,10.0,1.0,5,no
0,10.301221,10.114495,10.115675,10.115735,10.007082,10.127878,0.5,10.0,1.0,5,no


In [9]:
# Save raw coefficient results
output.to_csv(output_dir + '/n_' + str(N) + '_results.csv')

In [10]:
pd.set_option('display.max_rows', 10)

In [11]:
output['pca_ape'] = (output['pca_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_coef_ape'] = (output['mismeasured_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_allvar_coef_ape'] = (output['mismeasured_allvar_coef'] - output['beta1']).abs()/output['beta1']
output['mismeasured_avg_coef_ape'] = (output['mismeasured_avg_coef'] - output['beta1']).abs()/output['beta1']
output['iv_coef_ape'] = (output['iv_coef'] - output['beta1']).abs()/output['beta1']

In [12]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).mean().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.242976,0.535887,0.535864,0.535341,0.996467,0.484676,0.464659,0.757024,0.464113,0.464136,0.515324
1,-0.9,1.0,1.0,5,yes,0.242601,0.376047,0.176532,0.336822,0.998970,0.300439,0.663178,0.757399,0.623953,0.823468,0.699561
2,-0.5,1.0,1.0,5,no,0.714094,0.894339,0.894326,0.894160,0.999330,0.873866,0.105840,0.285906,0.105661,0.105674,0.126134
3,-0.5,1.0,1.0,5,yes,0.714589,0.818292,0.634314,0.791630,0.999622,0.765596,0.208370,0.285411,0.181708,0.365686,0.234404
4,0.0,1.0,1.0,5,no,0.999695,0.999722,0.999711,0.999713,0.999878,0.999741,0.015658,0.017784,0.015667,0.015671,0.015946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,3.847974,2.814628,4.648065,3.079584,1.000067,3.345202,2.079584,2.847974,1.814628,3.648065,2.345202
18,0.5,10.0,1.0,5,no,10.284443,10.104523,10.104542,10.104699,9.999312,10.125195,0.010470,0.028444,0.010452,0.010454,0.012520
19,0.5,10.0,1.0,5,yes,10.284892,10.181281,10.364676,10.208078,9.998673,10.234415,0.020808,0.028489,0.018128,0.036468,0.023441
20,0.9,1.0,1.0,5,no,1.755371,1.460984,1.460980,1.461417,1.000228,1.512270,0.461417,0.755371,0.460984,0.460980,0.512270


In [13]:
output.groupby(['covariance','beta1','beta2','p','exp_of_var']).std().reset_index()

,covariance,beta1,beta2,p,exp_of_var,mismeasured_coef,mismeasured_allvar_coef,mismeasured_avg_coef,pca_coef,true_val_coef,iv_coef,pca_ape,mismeasured_coef_ape,mismeasured_allvar_coef_ape,mismeasured_avg_coef_ape,iv_coef_ape
0,-0.9,1.0,1.0,5,no,0.026501,0.034061,0.033991,0.033973,0.042281,0.032840,0.033973,0.026501,0.034061,0.033991,0.032840
1,-0.9,1.0,1.0,5,yes,0.025653,0.030611,0.026243,0.035135,0.042004,0.030031,0.035135,0.025653,0.030611,0.026243,0.030031
2,-0.5,1.0,1.0,5,no,0.022976,0.022550,0.022550,0.022544,0.021774,0.022881,0.022544,0.022976,0.022550,0.022550,0.022881
3,-0.5,1.0,1.0,5,yes,0.023109,0.022223,0.028880,0.027002,0.020527,0.023826,0.027002,0.023109,0.022223,0.028880,0.023826
4,0.0,1.0,1.0,5,no,0.022385,0.019534,0.019543,0.019527,0.017928,0.019910,0.011660,0.013587,0.011659,0.011670,0.011914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0.5,1.0,10.0,5,yes,0.131514,0.112312,0.218805,0.191309,0.021204,0.135715,0.191309,0.131514,0.112312,0.218805,0.135715
18,0.5,10.0,1.0,5,no,0.023220,0.021595,0.021608,0.021614,0.020190,0.021876,0.002161,0.002322,0.002159,0.002161,0.002188
19,0.5,10.0,1.0,5,yes,0.023293,0.022921,0.028843,0.026722,0.020530,0.024268,0.002672,0.002329,0.002292,0.002884,0.002427
20,0.9,1.0,1.0,5,no,0.024876,0.033051,0.033031,0.033023,0.041606,0.031983,0.033023,0.024876,0.033051,0.033031,0.031983


Code to produce LaTeX tables:

In [14]:
# Helper functions for formatting latex output
def std_parenth_padder(type_col, col_to_pad):
    if type_col == 'std':
        return('(' + col_to_pad + ')')
    else:
        return(col_to_pad)

def format_level_0(type_col, col_to_format):
    if type_col == 'std':
        return('')
    else:
        return(col_to_format)

In [15]:
# Pass this function a pandas dataframe of which to create the LaTeX file
# dataframe is the df
# filename is the save location
# index tells if you want the index saved to the file or not
def Minimal_Latex(dataframe, filename, index = False):

    # Open file
    with open(filename, "w") as f:
        
        # Start from the pandas
        corrected_table = dataframe.to_latex(header = False, index = index)

        # Strip all material that pandas inserts- the tabular environment and toprule and bottomrule
        corrected_table = re.sub(r'\\begin.+', '', corrected_table)
        corrected_table = re.sub(r'\\toprule', '', corrected_table)
        corrected_table = re.sub(r'\\bottomrule', '', corrected_table)
        corrected_table = re.sub(r'\\end.+', '', corrected_table)

        # Write to file and skip blank lines
        f.write(corrected_table.strip())

In [16]:
coef_names_mapper = {'mismeasured_coef':'Single Measurement', 'mismeasured_allvar_coef':'All Measurements', 'pca_coef':'PCA', 'mismeasured_avg_coef':'Average of Measurements', 'iv_coef':'Instrumental Variable'}
ape_names_mapper = {'mismeasured_coef_ape':'Single Measurement', 'mismeasured_allvar_coef_ape':'All Measurements', 'pca_ape':'PCA', 'mismeasured_avg_coef_ape':'Average of Measurements', 'iv_coef_ape':'Instrumental Variable'}

In [17]:
all_params = ['beta1', 'beta2', 'covariance', 'p']
pd.set_option('display.max_rows', 25)

def make_param_tables(dataframe, parameter, exp):

    if parameter != 'beta1':
        dataframe = dataframe.loc[(dataframe['beta1']==1)]
    if parameter != 'beta2':
        dataframe = dataframe.loc[(dataframe['beta2']==1)]
    if parameter != 'covariance':
        dataframe = dataframe.loc[(dataframe['covariance']==0.5)]
    if parameter != 'p':
        dataframe = dataframe.loc[(dataframe['p']==5)]
    dataframe = dataframe.loc[(dataframe['exp_of_var']==exp)]

    exp_mapper = {'yes':'exp', 'no':'no_exp'}

    dataframe = (dataframe.drop(columns = [variable for variable in all_params if variable != parameter])
                          .groupby(parameter)
                          .agg(['mean', 'std'])
                          .transpose()
                          .reset_index())

    print(dataframe)

    dataframe.columns = dataframe.columns.map(str)

    dataframe_coefs = (dataframe.query('level_0.str.contains("coef")')
                                .query('level_0 != "true_val_coef"')
                                .query('not level_0.str.contains("ape")')
                                .round(3)
                                .astype(str))

    for variable in dataframe_coefs.columns:
        if variable != 'level_0' and variable != 'level_1':
            dataframe_coefs[variable] = dataframe_coefs.apply(lambda x: std_parenth_padder(x['level_1'], x[variable]), axis = 1)

    dataframe_coefs['level_0'] = dataframe_coefs.apply(lambda x: format_level_0(x.level_1, x.level_0), axis = 1)

    dataframe_coefs = (dataframe_coefs.drop(columns = 'level_1')
                                    .replace(coef_names_mapper))

    Minimal_Latex(dataframe_coefs, tables_dir + '/' + parameter + '_coefs_' + exp_mapper[exp] + '.tex')

    dataframe_apes = (dataframe.query('level_0.str.contains("ape")')
                           .query('level_1 != "std"')
                           .drop(columns = 'level_1')
                           .replace(ape_names_mapper)
                           .set_index('level_0')
                           .multiply(100)
                           .round(1)
                           .astype(str)
                           .add('%')
                           .reset_index())

    Minimal_Latex(dataframe_apes, tables_dir + '/' + parameter + '_apes_' + exp_mapper[exp] + '.tex')

In [18]:
make_param_tables(output, 'p', 'yes')

p                       level_0 level_1         5        20        50
0              mismeasured_coef    mean  1.285958  1.285384  1.286121
1              mismeasured_coef     std  0.023444  0.023101  0.022505
2       mismeasured_allvar_coef    mean  1.181737  1.056096  1.025023
3       mismeasured_allvar_coef     std  0.022736  0.020989  0.020654
4          mismeasured_avg_coef    mean  1.365991  1.302937  1.287386
5          mismeasured_avg_coef     std  0.028835  0.027252  0.025542
6                      pca_coef    mean  1.208243  1.080903  1.055086
7                      pca_coef     std  0.026568  0.022555  0.021760
8                 true_val_coef    mean  0.999904  0.999757  1.000616
9                 true_val_coef     std  0.020885  0.020558  0.020416
10                      iv_coef    mean  1.234392  1.062037  1.035296
11                      iv_coef     std  0.023896  0.021049  0.020989
12                      pca_ape    mean  0.208243  0.080903  0.055129
13                  

In [19]:
make_param_tables(output, 'p', 'no')

p                       level_0 level_1         5        20        50
0              mismeasured_coef    mean  1.286266  1.287102  1.285265
1              mismeasured_coef     std  0.022816  0.023559  0.023888
2       mismeasured_allvar_coef    mean  1.105807  1.031883  1.012755
3       mismeasured_allvar_coef     std  0.021403  0.021474  0.020830
4          mismeasured_avg_coef    mean  1.105804  1.031849  1.012849
5          mismeasured_avg_coef     std  0.021403  0.021477  0.020619
6                      pca_coef    mean  1.106005  1.032031  1.013055
7                      pca_coef     std  0.021387  0.021467  0.020623
8                 true_val_coef    mean  1.000266  1.000727  0.999820
9                 true_val_coef     std  0.020368  0.021345  0.020546
10                      iv_coef    mean  1.126255  1.037099  1.023450
11                      iv_coef     std  0.021635  0.021658  0.020909
12                      pca_ape    mean  0.106005  0.033144  0.019402
13                  

In [20]:
make_param_tables(output, 'covariance', 'yes')

covariance                      level_0 level_1      -0.9      -0.5       0.0  \
0                      mismeasured_coef    mean  0.242601  0.714589  0.999983   
1                      mismeasured_coef     std  0.025653  0.023109  0.022526   
2               mismeasured_allvar_coef    mean  0.376047  0.818292  1.000120   
3               mismeasured_allvar_coef     std  0.030611  0.022223  0.020732   
4                  mismeasured_avg_coef    mean  0.176532  0.634314  1.000745   
5                  mismeasured_avg_coef     std  0.026243  0.028880  0.023393   
6                              pca_coef    mean  0.336822  0.791630  1.000447   
7                              pca_coef     std  0.035135  0.027002  0.021099   
8                         true_val_coef    mean  0.998970  0.999622  1.000106   
9                         true_val_coef     std  0.042004  0.020527  0.018372   
10                              iv_coef    mean  0.300439  0.765596  1.000295   
11                          

In [21]:
make_param_tables(output, 'covariance', 'no')

covariance                      level_0 level_1      -0.9      -0.5       0.0  \
0                      mismeasured_coef    mean  0.242976  0.714094  0.999695   
1                      mismeasured_coef     std  0.026501  0.022976  0.022385   
2               mismeasured_allvar_coef    mean  0.535887  0.894339  0.999722   
3               mismeasured_allvar_coef     std  0.034061  0.022550  0.019534   
4                  mismeasured_avg_coef    mean  0.535864  0.894326  0.999711   
5                  mismeasured_avg_coef     std  0.033991  0.022550  0.019543   
6                              pca_coef    mean  0.535341  0.894160  0.999713   
7                              pca_coef     std  0.033973  0.022544  0.019527   
8                         true_val_coef    mean  0.996467  0.999330  0.999878   
9                         true_val_coef     std  0.042281  0.021774  0.017928   
10                              iv_coef    mean  0.484676  0.873866  0.999741   
11                          

In [22]:
make_param_tables(output, 'beta1', 'yes')

beta1                      level_0 level_1       0.1       1.0       10.0
0                 mismeasured_coef    mean  0.384904  1.285958  10.284892
1                 mismeasured_coef     std  0.023244  0.023444   0.023293
2          mismeasured_allvar_coef    mean  0.281532  1.181737  10.181281
3          mismeasured_allvar_coef     std  0.022321  0.022736   0.022921
4             mismeasured_avg_coef    mean  0.465073  1.365991  10.364676
5             mismeasured_avg_coef     std  0.027800  0.028835   0.028843
6                         pca_coef    mean  0.308825  1.208243  10.208078
7                         pca_coef     std  0.027099  0.026568   0.026722
8                    true_val_coef    mean  0.099577  0.999904   9.998673
9                    true_val_coef     std  0.020597  0.020885   0.020530
10                         iv_coef    mean  0.334748  1.234392  10.234415
11                         iv_coef     std  0.023492  0.023896   0.024268
12                         pca_ape    

In [23]:
make_param_tables(output, 'beta1', 'no')

beta1                      level_0 level_1       0.1       1.0       10.0
0                 mismeasured_coef    mean  0.386315  1.286266  10.284443
1                 mismeasured_coef     std  0.023860  0.022816   0.023220
2          mismeasured_allvar_coef    mean  0.205718  1.105807  10.104523
3          mismeasured_allvar_coef     std  0.022504  0.021403   0.021595
4             mismeasured_avg_coef    mean  0.205709  1.105804  10.104542
5             mismeasured_avg_coef     std  0.022572  0.021403   0.021608
6                         pca_coef    mean  0.205887  1.106005  10.104699
7                         pca_coef     std  0.022573  0.021387   0.021614
8                    true_val_coef    mean  0.100347  1.000266   9.999312
9                    true_val_coef     std  0.021496  0.020368   0.020190
10                         iv_coef    mean  0.226120  1.126255  10.125195
11                         iv_coef     std  0.022562  0.021635   0.021876
12                         pca_ape    

In [24]:
make_param_tables(output, 'beta2', 'yes')

beta2                      level_0 level_1       0.1       1.0      10.0
0                 mismeasured_coef    mean  1.028945  1.285958  3.847974
1                 mismeasured_coef     std  0.019375  0.023444  0.131514
2          mismeasured_allvar_coef    mean  1.018415  1.181737  2.814628
3          mismeasured_allvar_coef     std  0.019874  0.022736  0.112312
4             mismeasured_avg_coef    mean  1.036938  1.365991  4.648065
5             mismeasured_avg_coef     std  0.018851  0.028835  0.218805
6                         pca_coef    mean  1.021105  1.208243  3.079584
7                         pca_coef     std  0.019789  0.026568  0.191309
8                    true_val_coef    mean  1.000513  0.999904  1.000067
9                    true_val_coef     std  0.020917  0.020885  0.021204
10                         iv_coef    mean  1.023673  1.234392  3.345202
11                         iv_coef     std  0.019424  0.023896  0.135715
12                         pca_ape    mean  0.02384

In [25]:
make_param_tables(output, 'beta2', 'no')

beta2                      level_0 level_1       0.1       1.0      10.0
0                 mismeasured_coef    mean  1.028305  1.286266  3.848312
1                 mismeasured_coef     std  0.019732  0.022816  0.130960
2          mismeasured_allvar_coef    mean  1.010399  1.105807  2.049166
3          mismeasured_allvar_coef     std  0.020477  0.021403  0.085396
4             mismeasured_avg_coef    mean  1.010421  1.105804  2.049212
5             mismeasured_avg_coef     std  0.020446  0.021403  0.085309
6                         pca_coef    mean  1.010433  1.106005  2.050965
7                         pca_coef     std  0.020441  0.021387  0.085411
8                    true_val_coef    mean  0.999993  1.000266  0.999398
9                    true_val_coef     std  0.021121  0.020368  0.021442
10                         iv_coef    mean  1.012540  1.126255  2.253544
11                         iv_coef     std  0.020300  0.021635  0.091514
12                         pca_ape    mean  0.01852